In [94]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import numpy as np


In [12]:
def readCsv(filename):
    with open(filename) as f:  
        df = pd.read_csv(f, names=['UserID', 'MovieID'])
    return df

In [38]:
df = readCsv('user_movie_likes.csv')

In [39]:
df = df.set_index('MovieID')

In [40]:
df.head()

,UserID
MovieID,
512,412
770,458
37,185
701,137
870,190


In [41]:
df = pd.get_dummies(df.UserID)

In [42]:
df = df.sort_index()

In [43]:
df = df.groupby(df.index).sum()

In [68]:
nbrs = NearestNeighbors(n_neighbors=2).fit(df, df.index)

In [69]:
print(nbrs)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=1, n_neighbors=2, p=2, radius=1.0)


In [71]:
 print(nbrs.kneighbors(df, 3, False))

[[  0  30 428]
 [194   1  47]
 [ 65   2  18]
 ..., 
 [683 270  38]
 [684  75  44]
 [685 644 206]]


In [172]:
def getNearestThreeForMovie(filename, movieID):
    df = pd.read_csv(filename, names=['UserID', 'MovieID'])
    df = df.set_index('MovieID')
    df = pd.get_dummies(df.UserID)
    df = df.sort_index()
    df = df.groupby(df.index).sum()
    nbrs = NearestNeighbors().fit(df, df.index)
    return nbrs.kneighbors(df.loc[movieID, :].reshape(1,-1), 3, False)
    

In [173]:
getNearestThreeForMovie('user_movie_likes.csv', 993)

array([[681, 347,   6]])

In [180]:
def getNearestThreeForUser(filename, userID):
    df = pd.read_csv(filename, names=['UserID', 'MovieID'])
    df = df[df.UserID == userID]
    df = df.set_index('UserID')
    movieIDs = list(df.MovieID)
    recommended_movies = []
    for movie in movieIDs:
        recommended_movies.append(getNearestThreeForMovie(filename, movie)[0][0])
    #print(movieIDs)    
    #print(recommended_movies)
    unseen_movies = [movie for movie in recommended_movies if movie not in movieIDs]
    if len(unseen_movies) >= 3:
        print(unseen_movies[:3])
    else:
        print('Less than three results returned')

In [181]:
getNearestThreeForUser('user_movie_likes.csv', 412)

[352, 304, 682]
